 **Laplacian of Gaussian, Scale-Space Approach**

In [1]:
from matplotlib import pyplot as plt
from Functions import blob
import h5py
import os
import numpy as np
import datetime
from tifffile import imread
from tqdm import tqdm_notebook as tqdm


/home/manan/Softwares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**User-Defined Parameters**

In [2]:
mode='conservative' # conservative|liberal
datapath='/home/manan/Desktop/01_Projects/25_Tomas_GradientLOG/01_Data/BM_dataset_MICCAI2015/source/' #'/home/manan/Desktop/03_Datasets/13-03-12/processed_movie_hdf5_xml/' \ '/home/manan/Desktop/03_Datasets/24hpf/raw/'
dataname='source' # 13-03-12 | 13-03-15 | 24hpf
datatype='2D' # 2D | 3D
datamodality='fixed'# liveimaging | fixed
radiusmin=8
radiusmax=20
z_x=1 # 5 for liveimaging | 1 for fixed
y_x=1
brightondark= False # True for bright blobs on dark background, False for dark blobs on bright background

** User-Defined Parameters (Continued)**

In [3]:
outputaddress='/home/manan/Desktop/01_Projects/25_Tomas_GradientLOG/03_Output/'
headerdata='mode\t'+ mode +'\n'+'datapath\t'+datapath+'\n'+'dataname\t'+dataname+'\n'+'datatype\t'+datatype+'\n'+'datamodality\t'+datamodality+'\n'+'radiusmin\t'+str(radiusmin)+'\n'+'radiusmax\t'+str(radiusmax)+'\n'+'z_x\t'+str(z_x)+'\n'+'y_x\t'+str(y_x) + '\n'+ 'brightondark\t' +str(brightondark)

**Preprocessing** 

In [4]:
from numpy import sqrt
if datatype=='2D':
    scalemin=int(round(radiusmin/sqrt(2)))
    scalemax=int(round(radiusmax/sqrt(2)))
elif datatype=='3D':
    scalemin=int(round(radiusmin/sqrt(3)))
    scalemax=int(round(radiusmax/sqrt(3)))
scalerange=range(scalemin, scalemax)
plt.close('all')
now=datetime.datetime.now()
if os.path.isdir(outputaddress + '/' + dataname + '/'):
    if os.path.isdir(outputaddress + '/' +dataname + '/' + str(now.day)+str(now.month)+str(now.year) ):
        if os.path.isdir(outputaddress + '/' +dataname + '/' + str(now.day)+str(now.month)+str(now.year) +'/'+'01_Blobs/'):
            print("")
        else:
            os.mkdir(outputaddress + '/'+dataname + '/' + str(now.day)+str(now.month)+str(now.year)+'/'+'01_Blobs/')

    else:
        os.mkdir(outputaddress + '/'+dataname + '/' + str(now.day)+str(now.month)+str(now.year))
        os.mkdir(outputaddress + '/'+dataname + '/' + str(now.day)+str(now.month)+str(now.year)+'/'+'01_Blobs/')

else:
    os.mkdir(outputaddress + '/'+dataname)
    os.mkdir(outputaddress + '/'+dataname + '/' + str(now.day)+str(now.month)+str(now.year))
    os.mkdir(outputaddress + '/'+dataname + '/' + str(now.day)+str(now.month)+str(now.year)+'/'+'01_Blobs/')


**Computation begins**: Let's obtain features (**~nuclei centroids**) and save predictions 

In [5]:
if datamodality == 'liveimaging':
    for filename in tqdm(os.listdir(datapath)):
        if (filename=='dataset_hdf5.h5'):
            f = h5py.File(datapath + filename, 'r')
            for timestep in reversed(list(f)):
                if ((timestep[0]=='t') and (timestep=='t00600')):
                    data = f[timestep +'/s00/0/cells']
                    data=np.float32(data)
                    timevalue=timestep[-3:]
                    if(brightondark):
                        blobs = blob.findBlobs(data, datatype, z_x, y_x, mode, scalerange)
                    else:
                        blobs = blob.findBlobs(-data, datatype, z_x, y_x, mode, scalerange)
                    np.savetxt(outputaddress+'/'+dataname+'/'+str(now.day)+str(now.month)+str(now.year) +'/01_Blobs/'+timestep+'.csv', blobs, header=headerdata, delimiter='\t')
elif datamodality == 'fixed':
    for filename in tqdm(os.listdir(datapath)):
        data=plt.imread(datapath+filename).astype('float32').sum(axis=-1)
        if (brightondark):
            blobs = blob.findBlobs(data, datatype, z_x, y_x, mode, scalerange)
        else:
            blobs = blob.findBlobs(-data, datatype, z_x, y_x, mode, scalerange)
        np.savetxt(outputaddress+dataname+'/'+str(now.day)+str(now.month)+str(now.year) +'/01_Blobs/'+filename[:-4]+'.csv', blobs, header=headerdata, delimiter='\t')            

/home/manan/Desktop/01_Projects/25_Tomas_GradientLOG/02_Notebook/Functions/blob.py:96: RuntimeWarning: divide by zero encountered in true_divide
  - sqrt((-d + r1 + r2) * (d + r1 - r2)


**Congrats**, Feature Detectors have beeen computed.
Next, access **02_Visualize_Feature_Detectors** to visualize predictions

In [6]:
print(filename)

BM_GRAZ_HE_0020_01.png
